The first order gas phase reaction where $A \rightarrow B$ is performed in a packed bed reactor. You can assume the absence of external mass transfer effects. Given the information below determine the intrinsic rate constant at this temperature ($k_{i}^{’}$).

($k_{i}^{’}= 1.009 × 10^{−4}~m^{3}/(kg.s)$, $\eta = 0.44$)

<img src="Tut5.2 data.JPG" width="450" />

In [1]:
from numpy import *
from scipy.optimize import fsolve
from scipy.integrate import odeint
from matplotlib import pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline

Since it is stated that external mass transfer can be assumed absent, it follows that $C_{A_{b}} = C_{A_{s}}$

Therefore $$\dfrac{1}{k_{eff}^{'}} = \dfrac{1}{\eta.k_{i}^{'}}$$
$$k_{eff}^{'} = \eta.k_{i}^{'}$$

In [2]:
W      =             7 #kg
dp     =        2/1000 #m
R      =          dp/2 #m partical radius
rhocat =          1600 #kg/m3
De     =          5e-9 #m2/s
Q      =  20/(1000*60) #L/s
X      =          0.60
epsilon=           0.4 #porosity
rho    =          1100 #kg/m3 liquid density
n      =             1 # 1st order reaction

# PBR - Mole Balance

For plug flow

$$
\begin{align}
\frac{dF_A}{dW}&=-r_A'\\
\frac{dF_A}{dW}&=\eta k_{i}^{'}C_A\\
QC_{Ao}\frac{dx}{dW}&=\eta k_{i}^{'}C_{Ao}(1-x)\\
\int^{x=0.6}_{x=0}\frac{dx}{1-x}&=\frac{\eta k_{i}^{'}W}{Q}\\
-\ln(1-x)\big\rvert^{0.6}_0 &=\frac{\eta k_{i}W}{Q}\\
\eta k_{i} &=-\frac{Q\ln(1-x)\big\rvert^{0.6}_0}{W} \\
-ln(1 - x) &= \eta k_{i}^{'}\dfrac{W}{Q}  
\end{align}
$$

# Effectiveness ($\eta$) - First Order Reaction

$$
\eta=\frac{3}{\phi_1^2}\left(\phi_1\coth\phi_1-1\right)
$$

# Thiele-Modulus ($\phi_n$)

\begin{align}
\phi_n^2&=\frac{k_i'\rho_cR^2C_{As}^{n-1}}{D_e}\\
&=\frac{\frac{k_{app}}{\eta}\rho_cR^2C_{As}^{n-1}}{D_e}\\
\phi_n&=\left(\frac{k_i'\rho_cR^2}{D_e} \right)^{0.5}
\end{align}

Remember: $\eta \leq 1$ 

First method is to do a fixed point iteration between the mole balance, eta and phi equations by guessing $\eta$ to be 1 in order to calculate $k_{i}^{'}$, taking this result and calculating $\phi$ then using this value to calculate the new $\eta$. This iteration is continued until $\eta$ converges, then in turn calculating the final $\phi$

In [3]:
eta   = 1 #first guess
error = 0.1 #to get iteration started

In [4]:
while error >= 0.0000000001:
    kiprime  = -log(1 - X)*Q/(eta*W) #mole balance equation
    phi      = ((kiprime*rhocat*(R)**2)/De)**0.5 #Thiele modules equation
    etanew   = 3*((phi/tanh(phi)) - 1)/(phi**2) #Effectiveness equation
    error    = abs(eta - etanew)
    eta      = etanew

In [5]:
print('$k_{i}^{\'}$ = '+str(kiprime))
print('$\eta$ = '+str(eta))

$k_{i}^{'}$ = 9.990039717711863e-05
$\eta$ = 0.4367639490734235


Second method is to use fsolve to calculate $\eta$ and $k_{i}^{'}$ (first two guesses), and using the Thiele equation as a variable. 

In [6]:
def PET(var):
    kiprime, eta = var
    phi = ((1.00915621e-04*rhocat*(R)**2)/De)**0.5 #use kiprime result from fsolve and replace here until it converges
    EQ1 = -log(1 - X) - eta*kiprime*W/Q
    EQ2 = 3*((phi/tanh(phi)) - 1)/(phi**2) - eta
    return [EQ1, 
            EQ2]

In [7]:
PET([9.990039717711863e-05, 0.4367639490734235])


[1.874368438237184e-10, -0.0017323799528450867]

In [8]:
guess = [1, 2]
sols  = fsolve(PET, guess)
sols

array([1.00298220e-04, 4.35031569e-01])